In [ ]:
import streamlit as st
import pytesseract
import pyttsx3
import cv2
import numpy as np
from PIL import Image
from langchain import PromptTemplate, LLMChain
from langchain.chat_models import ChatOpenAI

# Initialize Text-to-Speech engine
engine = pyttsx3.init()

# Streamlit App Configuration
st.set_page_config(page_title="AI Assist for Visually Impaired", layout="wide")
st.title("🧑‍🦯 AI Assistant for Visually Impaired")

# Load LangChain Model
llm = ChatOpenAI(model="gpt-3.5-turbo")  # Replace with a valid API model
template = """
You are an AI assistant helping visually impaired users. Here is the scene description:
{description}
Respond in a clear, friendly manner.
"""
prompt = PromptTemplate(template=template, input_variables=["description"])
chain = LLMChain(llm=llm, prompt=prompt)

# Function for scene description
def describe_scene(image):
    # Simulate object detection (replace with your model integration)
    # Using a placeholder for detected objects
    detected_objects = ["a chair", "a table", "a person"]
    description = f"I see {', '.join(detected_objects)} in front of you."
    return description

# Function to extract text from an image
def read_text(image):
    text = pytesseract.image_to_string(image)
    return text if text.strip() else "No readable text detected."

# Function to convert text to speech
def speak_text(text):
    engine.say(text)
    engine.runAndWait()

# Sidebar Options
st.sidebar.header("Options")
tts_enabled = st.sidebar.checkbox("Enable Text-to-Speech", value=True)

# Upload or capture image
st.header("Upload or Capture an Image")
uploaded_file = st.file_uploader("Choose an image...", type=["jpg", "png", "jpeg"])

if uploaded_file:
    # Display the uploaded image
    image = Image.open(uploaded_file)
    st.image(image, caption="Uploaded Image", use_column_width=True)

    # Convert to OpenCV format for processing
    image_np = np.array(image)

    # Scene description
    st.subheader("Scene Understanding")
    scene_description = describe_scene(image_np)
    st.write(scene_description)

    # Use LangChain to enhance interaction
    enhanced_response = chain.run(description=scene_description)
    st.write("🤖 Enhanced Assistant Response:")
    st.write(enhanced_response)

    # Optional Text-to-Speech
    if tts_enabled:
        speak_text(enhanced_response)

    # Text extraction
    st.subheader("Text Recognition")
    extracted_text = read_text(image)
    st.write(extracted_text)

    if tts_enabled and extracted_text != "No readable text detected.":
        speak_text(f"Detected text: {extracted_text}")
else:
    st.write("Please upload an image to analyze.")

# Camera Input
st.header("Real-Time Camera Feed")
camera_enabled = st.checkbox("Enable Camera")

if camera_enabled:
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        st.error("Unable to access the camera.")
    else:
        ret, frame = cap.read()
        if ret:
            st.image(frame, channels="BGR", caption="Real-Time Camera Feed")
            # Process frame for scene understanding and text
            scene_description = describe_scene(frame)
            st.write(f"Scene: {scene_description}")

            if tts_enabled:
                speak_text(scene_description)
else:
    st.write("Enable the camera to capture real-time input.")
